# Named entity recognition with OpenVINO™


The Named Entity Recognition(NER) is a natural language processing method that involves the detecting of key information in the unstructured text and categorizing it into pre-defined categories. These categories or named entities refer to the key subjects of text, such as names, locations, companies and etc.

NER is a good method for the situations when a high-level overview of a large amount of text is needed. NER can be helpful with such task as analyzing key information in unstructured text or automates the information extraction of large amounts of data.


This tutorial shows how to perform named entity recognition using OpenVINO. We will use the pre-trained model [elastic/distilbert-base-cased-finetuned-conll03-english](https://huggingface.co/elastic/distilbert-base-cased-finetuned-conll03-english). It is DistilBERT based model, trained on [conll03 english dataset](https://huggingface.co/datasets/conll2003). The model can recognize four named entities in text: persons, locations, organizations and names of miscellaneous entities that do not belong to the previous three groups. The model is sensitive to capital letters.

To simplify the user experience, the [Hugging Face Optimum](https://huggingface.co/docs/optimum) library is used to convert the model to OpenVINO™ IR format and quantize it.


Tutorial consists of the following steps:
- Download the model
- Quantize and save the model in OpenVINO IR format
- Prepare demo for Named Entity Recognition OpenVINO Runtime
- Compare the Original and Quantized Models

## Prerequisites 

In [ ]:
! pip install --upgrade -q "diffusers==0.17.1" "openvino-dev==2023.0.0" "nncf==2.5.0" "gradio" "onnx>=1.11.0" "onnxruntime>=1.14.0" "optimum-intel>=1.9.1" 

## Download the NER model

We load the [distilbert-base-cased-finetuned-conll03-english](https://huggingface.co/elastic/distilbert-base-cased-finetuned-conll03-english) model from the [Hugging Face Hub](https://huggingface.co/models) with [Hugging Face Transformers library](https://huggingface.co/docs/transformers/index).

Model class initialization starts with calling `from_pretrained` method. To easily save the model, you can use the `save_pretrained()` method.

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_id = "elastic/distilbert-base-cased-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(model_id)

original_ner_model_dir = 'original_ner_model'
model.save_pretrained(original_ner_model_dir)

tokenizer = AutoTokenizer.from_pretrained(model_id)

## Quantize the model, using Hugging Face Optimum API

Post-training static quantization introduces an additional calibration step where data is fed through the network in order to compute the activations quantization parameters. For quantization it will be used [Hugging Face Optimum Intel API](https://huggingface.co/docs/optimum/intel/index).

To handle the NNCF quantization process we use class [OVQuantizer](https://huggingface.co/docs/optimum/intel/reference_ov#optimum.intel.OVQuantizer). The quantization with Hugging Face Optimum Intel API contains the next steps:
* Model class initialization starts with calling `from_pretrained()` method.
* Next we create calibration dataset with `get_calibration_dataset()` to use for the post-training static quantization calibration step. 
* After we quantize a model and save the resulting model in the OpenVINO IR format to save_directory with `quantize()` method. 
* Then we load the quantized model. The Optimum Inference models are API compatible with Hugging Face Transformers models and we can just replace `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. So we use `OVModelForTokenClassification` to load the model.

In [ ]:
from functools import partial
from optimum.intel import OVQuantizer

from optimum.intel import OVModelForTokenClassification

def preprocess_fn(data, tokenizer):
    examples = []
    for data_chunk in data["tokens"]:
        examples.append(' '.join(data_chunk))

    return tokenizer(
        examples, padding=True, truncation=True, max_length=128
    )

quantizer = OVQuantizer.from_pretrained(model)
calibration_dataset = quantizer.get_calibration_dataset(
    "conll2003",
    preprocess_function=partial(preprocess_fn, tokenizer=tokenizer),
    num_samples=100,
    dataset_split="train",
    preprocess_batch=True,
)

# The directory where the quantized model will be saved
quantized_ner_model_dir = "quantized_ner_model"

# Apply static quantization and save the resulting model in the OpenVINO IR format
quantizer.quantize(calibration_dataset=calibration_dataset, save_directory=quantized_ner_model_dir)

# Load the quantized model
optimized_model = OVModelForTokenClassification.from_pretrained(quantized_ner_model_dir)

## Prepare demo for Named Entity Recognition OpenVINO Runtime

As the Optimum Inference models are API compatible with Hugging Face Transformers models, we can just use `pipleine()` from [Hugging Face Transformers API](https://huggingface.co/docs/transformers/index) for inference.

In [ ]:
from transformers import pipeline

ner_pipeline_optimized = pipeline("token-classification", model=optimized_model, tokenizer=tokenizer)

Now, you can try NER model on own text. Put your sentence to input text box, click Submit button, the model label the recognized entities in the text.

In [ ]:
import gradio as gr

examples = [
    "My name is Wolfgang and I live in Berlin.",
]

def run_ner(text):
    output = ner_pipeline_optimized(text)
    return {"text": text, "entities": output} 

demo = gr.Interface(run_ner,
                    gr.Textbox(placeholder="Enter sentence here...", label="Input Text"), 
                    gr.HighlightedText(label="Output Text"),
                    examples=examples,
                    allow_flagging="never")

if __name__ == "__main__":
    try:
        demo.launch(debug=True)
    except Exception:
        demo.launch(share=True, debug=True)
# if you are launching remotely, specify server_name and server_port
# demo.launch(server_name='your server name', server_port='server port in int')
# Read more in the docs: https://gradio.app/docs/

## Compare the Original and Quantized Models

Compare the original [distilbert-base-cased-finetuned-conll03-english](https://huggingface.co/elastic/distilbert-base-cased-finetuned-conll03-english) model with quantized and converted to OpenVINO IR formant models to see the difference.

### Compare performance

In [ ]:
ner_pipeline_original = pipeline("token-classification", model=model, tokenizer=tokenizer)

In [ ]:
import time
import numpy as np

def calc_perf(ner_pipeline):
    inference_times = []

    for data in calibration_dataset:
        text = ' '.join(data['tokens'])
        start = time.perf_counter()
        ner_pipeline(text)
        end = time.perf_counter()
        inference_times.append(end - start)

    return np.median(inference_times)


print(
    f"Median inference time of quantized model: {calc_perf(ner_pipeline_optimized)} "
)

print(
    f"Median inference time of original model: {calc_perf(ner_pipeline_original)} "
)

### Compare size of the models

In [ ]:
from pathlib import Path

print(f'Size of original model in Bytes is {Path(original_ner_model_dir, "pytorch_model.bin").stat().st_size}')
print(f'Size of quantized model in Bytes is {Path(quantized_ner_model_dir, "openvino_model.bin").stat().st_size}')